Notebook to use logistic regression on the Animal Future survey data to predict farmers adoption.

In [3]:
import pandas as pd
import numpy as np

# Data preparation

## Data upload

In [4]:
path_to_data = "./survey_data/AF_survey_data_30.xlsx"

In [5]:
dataset_original = pd.read_excel(path_to_data, index_col=0)
dataset_original.head()

,AdoptedSBP,PastureSurface,CattlePercentage,Distrito,Concelho,FarmerSince,PercentRentedLand,LegalForm,HighestEducationalDegree,HighestAgriculturalEducationalDegree,ExpectationFamilySuccession
FARM_ID,,,,,,,,,,,
PT02,0,364.00,0.0,Setúbal,Grândola,29,0.0,Individual,Undergraduate,Undergraduate,Yes
PT13,1,542.58,1.0,Portalegre,Avis,11,0.0,Associated,Undergraduate,Undergraduate,Yes
PT15,1,262.70,1.0,Portalegre,Monforte,11,1.0,Associated,Undergraduate,Undergraduate,Yes
PT16,0,23.00,1.0,Évora,Évora,3,1.0,Individual,Undergraduate,Undergraduate,Yes
PT17,1,250.00,1.0,Évora,Montemor,10,1.0,Associated,Undergraduate,None,Yes


## Features reduction

In [6]:
features_to_drop = ['Concelho', 'HighestAgriculturalEducationalDegree']

In [7]:
dataset = dataset_original.drop(features_to_drop, axis=1)

In [8]:
dataset.head()

,AdoptedSBP,PastureSurface,CattlePercentage,Distrito,FarmerSince,PercentRentedLand,LegalForm,HighestEducationalDegree,ExpectationFamilySuccession
FARM_ID,,,,,,,,,
PT02,0,364.00,0.0,Setúbal,29,0.0,Individual,Undergraduate,Yes
PT13,1,542.58,1.0,Portalegre,11,0.0,Associated,Undergraduate,Yes
PT15,1,262.70,1.0,Portalegre,11,1.0,Associated,Undergraduate,Yes
PT16,0,23.00,1.0,Évora,3,1.0,Individual,Undergraduate,Yes
PT17,1,250.00,1.0,Évora,10,1.0,Associated,Undergraduate,Yes


## Data preparation

In [9]:
dataset_att = dataset.drop('AdoptedSBP', axis=1)
labels = dataset['AdoptedSBP'].copy()

### Categorical attributes

Ordinal encoding

In [10]:
ordinal_attributes = ['HighestEducationalDegree']

In [11]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder_education = OrdinalEncoder(categories=[['Primary', 'Secondary', 'Undergraduate', 'Graduate']])

One-hot encoding

In [12]:
onehot_attributes = ['Distrito']

In [13]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()
onehot_encoder.fit_transform(dataset_att[onehot_attributes])

<30x5 sparse matrix of type '<class 'numpy.float64'>'
	with 30 stored elements in Compressed Sparse Row format>

Binary attributes

In [14]:
binary_attributes = ['ExpectationFamilySuccession', 'LegalForm']

### Numerical attributes

In [15]:
dataset_att.columns

Index(['PastureSurface', 'CattlePercentage', 'Distrito', 'FarmerSince',
       'PercentRentedLand', 'LegalForm', 'HighestEducationalDegree',
       'ExpectationFamilySuccession'],
      dtype='object')

In [16]:
numerical_attributes = [feat for feat in dataset_att.columns if (
    (feat not in ordinal_attributes) and (feat not in onehot_attributes) and (feat not in binary_attributes)
)]

In [17]:
numerical_attributes

['PastureSurface', 'CattlePercentage', 'FarmerSince', 'PercentRentedLand']

### Preparation pipeline

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [19]:
preparation_pipeline = ColumnTransformer([
    ('num', StandardScaler(), numerical_attributes),
    ('ord_cat_edu', ordinal_encoder_education, ['HighestEducationalDegree']),
    ('other_ord_cat', OrdinalEncoder(), binary_attributes),
    ('onehot_cat', onehot_encoder, onehot_attributes)
])

In [20]:
dataset_prep = preparation_pipeline.fit_transform(dataset_att)

### Extract all attributes

In [21]:
attributes = (numerical_attributes
              + ['HighestEducationalDegree']
              + binary_attributes)
for cat_name in onehot_encoder.categories_:
    attributes += cat_name.tolist()

In [22]:
# Print prepared data as a DataFrame
pd.DataFrame(dataset_prep, columns=attributes, index=dataset_att.index).head()

,PastureSurface,CattlePercentage,FarmerSince,PercentRentedLand,HighestEducationalDegree,ExpectationFamilySuccession,LegalForm,Beja,Portalegre,Santarém,Setúbal,Évora
FARM_ID,,,,,,,,,,,,
PT02,-0.239190,-2.378406,1.146256,-0.615057,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
PT13,0.054486,0.556318,-0.444948,-0.615057,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
PT15,-0.405778,0.556318,-0.444948,1.858870,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
PT16,-0.799967,0.556318,-1.152150,1.858870,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
PT17,-0.426663,0.556318,-0.533348,1.858870,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


# Logistic regression

## Functions for model analysis

In [23]:
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics import f1_score, precision_score, recall_score

In [24]:
def display_scores(scores, score_name):
    #print(score_name + ' scores:', scores)
    print(score_name + ' mean:', scores.mean())
    #print(score_name + ' stdv:', scores.std())

In [25]:
def cross_val_scores(clsf, dataset_prepared, labels, seed):
    metrics = ['f1', 'precision', 'recall']
    metric_names = ['F1 score', 'precision', 'recall']
    scores = cross_validate(clsf, dataset_prepared, labels,
                            scoring=metrics, 
                            cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=seed))
    
    cv_scores = {}
    for (name, metric)  in zip(metric_names, metrics):
        cv_scores[name] = scores['test_' + metric]
    return cv_scores

In [26]:
def predict_train_set(clsf, dataset_prepared, labels):
    clsf.fit(dataset_prepared, labels)
    pred_probs = clsf.predict_proba(dataset_prepared)[:, 1]
    pred_classes = pred_probs >= 0.5
    f1 = f1_score(labels, pred_classes)
    prec = precision_score(labels, pred_classes)
    rec = recall_score(labels, pred_classes)

    print('Prediction on training set results')
    print('F1 score:', f1)
    print('Precision:', prec)
    print('Recall:', rec)

In [27]:
from sklearn.base import clone

def test_classifier(clsf, dataset_prepared, labels, seed=None):  
    """
    Function to:
    - train a classifier using cross validation, reporting performance measures
    - get classifier's predictions on the training set, to check it for overfiting
    """
    clsf_copy = clone(clsf) 
    cv_scores = cross_val_scores(clsf_copy, dataset_prepared, labels, seed)
    print("Cross validation scores")
    for score_name, score_value in cv_scores.items():
        display_scores(score_value, score_name)
    print("")
    
    train_scores = predict_train_set(clsf_copy, dataset_prepared, labels)

## All features

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import reciprocal, uniform

In [41]:
cross_val_split = StratifiedKFold(n_splits=3, shuffle=True)

In [66]:
# Nested cross-validation to try different split into train and test set
log_reg = LogisticRegression()

param_grid = {
    #"tol": [5e-4],
    "max_iter": [5000],
    "penalty": ["elasticnet"],
    "solver": ["saga"],
    "l1_ratio": uniform(0., 1.),
    "C": uniform(0.001, 1)
    }

NUM_TRIALS = 30
rnd_srch_results = {}
for i in range(NUM_TRIALS):
     cv_split = StratifiedKFold(n_splits=3, shuffle=True, random_state=i)
     rnd_srch = RandomizedSearchCV(log_reg, param_grid, cv=cv_split, n_iter=1000,
                                  scoring='f1', return_train_score=True, verbose=0)
     rnd_srch.fit(dataset_prep, labels)
     rnd_srch_results[rnd_srch.best_estimator_] = rnd_srch.best_score_
     print("Iteration number", str(i), "completed.")

Iteration number 0 completed.
Iteration number 1 completed.
Iteration number 2 completed.
Iteration number 3 completed.
Iteration number 4 completed.
Iteration number 5 completed.
Iteration number 6 completed.
Iteration number 7 completed.
Iteration number 8 completed.
Iteration number 9 completed.
Iteration number 10 completed.
Iteration number 11 completed.
Iteration number 12 completed.
Iteration number 13 completed.
Iteration number 14 completed.
Iteration number 15 completed.
Iteration number 16 completed.
Iteration number 17 completed.
Iteration number 18 completed.
Iteration number 19 completed.
Iteration number 20 completed.
Iteration number 21 completed.
Iteration number 22 completed.
Iteration number 23 completed.
Iteration number 24 completed.
Iteration number 25 completed.
Iteration number 26 completed.
Iteration number 27 completed.
Iteration number 28 completed.
Iteration number 29 completed.


In [67]:
rnd_srch_results.values()

dict_values([0.7222222222222222, 0.7428571428571429, 0.738095238095238, 0.7222222222222222, 0.7222222222222222, 0.7222222222222222, 0.7388888888888889, 0.7222222222222222, 0.7222222222222222, 0.7222222222222222, 0.7222222222222222, 0.7222222222222222, 0.7388888888888889, 0.7269841269841271, 0.7222222222222222, 0.7222222222222222, 0.7222222222222222, 0.7388888888888889, 0.7222222222222222, 0.7222222222222222, 0.7222222222222222, 0.7222222222222222, 0.7261904761904762, 0.7222222222222222, 0.7388888888888889, 0.7388888888888889, 0.7222222222222222, 0.7388888888888889, 0.7388888888888889, 0.7222222222222222])

In [68]:
best_score = max(rnd_srch_results.values())
best_score

0.7428571428571429

In [69]:
best_models = [k for k, v in rnd_srch_results.items() if v == best_score]
best_models

[LogisticRegression(C=0.09856505069929578, l1_ratio=0.030985963187024845,
                    max_iter=5000, penalty='elasticnet', solver='saga')]

In [70]:
best_log_reg = best_models[0]

In [74]:
# Get index of the best model to obtain same split when evaluating
best_indexes = [i for i, v in enumerate(rnd_srch_results.values()) if v == best_score]
best_index = best_indexes[0]

In [123]:
pred = best_log_reg.predict(dataset_prep)
pred

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1], dtype=int64)

In [124]:
#Percentage of adopters
sum(pred) / len(pred)

0.8666666666666667

In [76]:
for attr, coef in zip(attributes, best_log_reg.coef_.tolist()[0]):
    print(coef, attr)

0.10129553902221193 PastureSurface
-0.015765681748275678 CattlePercentage
-0.00718077613298788 FarmerSince
-0.1301324879077995 PercentRentedLand
0.07470563477517995 HighestEducationalDegree
0.021328708700832963 ExpectationFamilySuccession
-0.086755980127019 LegalForm
-0.10353149447066375 Beja
0.1282481322449387 Portalegre
0.0021488692280678405 Santarém
0.0 Setúbal
-0.002750013406458137 Évora


In [80]:
test_classifier(best_log_reg, dataset_prep, labels, seed=best_index)

Cross validation scores
F1 score mean: 0.7428571428571429
precision mean: 0.6157407407407408
recall mean: 0.9444444444444445

Prediction on training set results
F1 score: 0.6976744186046512
Precision: 0.5769230769230769
Recall: 0.8823529411764706


## Reduced number of features

In [29]:
numerical_attributes_red = ['PastureSurface', 'PercentRentedLand']
binary_attributes_red = ['LegalForm']

In [30]:
preparation_pipeline_red = ColumnTransformer([
    ('num', StandardScaler(), numerical_attributes_red),
    ('ord_cat_edu', ordinal_encoder_education, ['HighestEducationalDegree']),
    ('other_ord_cat', OrdinalEncoder(), binary_attributes_red),
])

In [31]:
dataset_prep_red = preparation_pipeline_red.fit_transform(dataset_att)

In [32]:
attributes_red = (numerical_attributes_red + ['HighestEducationalDegree'] + binary_attributes_red)

In [85]:
# Nested cross-validation to try different split into train and test set
log_reg = LogisticRegression()

param_grid = {
    #"tol": [5e-4],
    "max_iter": [5000],
    "penalty": ["elasticnet"],
    "solver": ["saga"],
    "l1_ratio": uniform(0., 1.),
    "C": reciprocal(0.001, 1)
    }

NUM_TRIALS = 30
rnd_srch_results_red = {}
for i in range(NUM_TRIALS):
     cv_split = StratifiedKFold(n_splits=3, shuffle=True, random_state=i)
     rnd_srch = RandomizedSearchCV(log_reg, param_grid, cv=cv_split, n_iter=1000,
                                  scoring='f1', return_train_score=True, verbose=0)
     rnd_srch.fit(dataset_prep_red, labels)
     rnd_srch_results_red[rnd_srch.best_estimator_] = rnd_srch.best_score_
     print("Iteration number", str(i), "completed.")

Iteration number 0 completed.
Iteration number 1 completed.
Iteration number 2 completed.
Iteration number 3 completed.
Iteration number 4 completed.
Iteration number 5 completed.
Iteration number 6 completed.
Iteration number 7 completed.
Iteration number 8 completed.
Iteration number 9 completed.
Iteration number 10 completed.
Iteration number 11 completed.
Iteration number 12 completed.
Iteration number 13 completed.
Iteration number 14 completed.
Iteration number 15 completed.
Iteration number 16 completed.
Iteration number 17 completed.
Iteration number 18 completed.
Iteration number 19 completed.
Iteration number 20 completed.
Iteration number 21 completed.
Iteration number 22 completed.
Iteration number 23 completed.
Iteration number 24 completed.
Iteration number 25 completed.
Iteration number 26 completed.
Iteration number 27 completed.
Iteration number 28 completed.
Iteration number 29 completed.


In [128]:
rnd_srch_results_red.values()

dict_values([0.738095238095238, 0.7261904761904762, 0.738095238095238, 0.7222222222222222, 0.7428571428571429, 0.7222222222222222, 0.7388888888888889, 0.7222222222222222, 0.7222222222222222, 0.7222222222222222, 0.7428571428571429, 0.7222222222222222, 0.7388888888888889, 0.7472527472527473, 0.7222222222222222, 0.7222222222222222, 0.7222222222222222, 0.7269841269841271, 0.7509157509157509, 0.7222222222222222, 0.738095238095238, 0.7222222222222222, 0.7222222222222222, 0.7388888888888889, 0.7388888888888889, 0.7388888888888889, 0.7269841269841271, 0.7222222222222222, 0.7222222222222222, 0.7222222222222222])

In [144]:
best_score_red = max(rnd_srch_results_red.values())
best_score_red

0.7509157509157509

In [145]:
best_models_red = [k for k, v in rnd_srch_results_red.items() if v == best_score_red]
best_models_red

[LogisticRegression(C=0.912239862932476, l1_ratio=0.35646542116124347,
                    max_iter=5000, penalty='elasticnet', solver='saga')]

In [146]:
best_log_reg_red = best_models_red[0]

In [149]:
# Get index of the best model to obtain same split when evaluating
best_indexes = [i for i, v in enumerate(rnd_srch_results_red.values()) if v == best_score_red]
best_index = best_indexes[0]

In [151]:
pred = best_log_reg_red.predict(dataset_prep_red)
pred

array([1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1], dtype=int64)

In [152]:
#Percentage of adopters
sum(pred) / len(pred)

0.7333333333333333

In [153]:
for attr, coef in zip(attributes_red, best_log_reg_red.coef_.tolist()[0]):
    print(coef, attr)

0.14292229841822537 PastureSurface
-0.23624240843791325 PercentRentedLand
0.14133188610105643 HighestEducationalDegree
-0.3752760221423707 LegalForm


In [154]:
best_log_reg_red.intercept_

array([0.05109831])

In [155]:
test_classifier(best_log_reg_red, dataset_prep_red, labels, seed=best_index)

Cross validation scores
F1 score mean: 0.7509157509157509
precision mean: 0.6547619047619048
recall mean: 0.888888888888889

Prediction on training set results
F1 score: 0.6666666666666667
Precision: 0.5909090909090909
Recall: 0.7647058823529411


# Logistic regression selected on whole training set with 4 variables (same as FL Calibrated ABM)

In [112]:
from sklearn.model_selection import ParameterSampler
from sklearn.base import clone

def RandomizedSearch(estimator, param_distribution, n_iter, X, y):
    best_f1 = 0
    best_model = None
    for g in ParameterSampler(param_distribution, n_iter):
        estimator.set_params(**g)
        estimator.fit(X, y)
        # get score and save if best
        preds = estimator.predict(X)
        f1 = f1_score(y, preds)
        if f1 > best_f1:
            best_f1 = f1
            best_model = clone(estimator)
    best_model.fit(X, y)
    return (best_model, best_f1)

In [113]:
log_reg = LogisticRegression()

param_grid = {
    #"tol": [5e-4],
    "max_iter": [5000],
    "penalty": ["elasticnet"],
    "solver": ["saga"],
    "l1_ratio": uniform(0., 1.),
    "C": uniform(0.001, 1)
    }

best_log_reg_nocv, best_f1 = RandomizedSearch(log_reg, param_grid, 10000, dataset_prep_red, labels)

In [114]:
best_log_reg_nocv

LogisticRegression(C=0.04804814307242389, l1_ratio=0.010059536226641685,
                   max_iter=5000, penalty='elasticnet', solver='saga')

In [115]:
best_f1

0.7555555555555554

In [116]:
pred = best_log_reg_nocv.predict(dataset_prep_red)
pred

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [4]:
#Percentage of adopters
sum(pred) / len(pred)

0.9333333333333333

In [117]:
for attr, coef in zip(attributes_red, best_log_reg_nocv.coef_.tolist()[0]):
    print(coef, attr)

0.06834798895991871 PastureSurface
-0.08735236435874066 PercentRentedLand
0.056719808108618215 HighestEducationalDegree
-0.05312923709388828 LegalForm


In [118]:
best_log_reg_nocv.intercept_

array([0.15329722])

In [119]:
test_classifier(best_log_reg_nocv, dataset_prep_red, labels)

Cross validation scores
F1 score mean: 0.6626984126984127
precision mean: 0.5333333333333333
recall mean: 0.8888888888888888

Prediction on training set results
F1 score: 0.7555555555555554
Precision: 0.6071428571428571
Recall: 1.0


# Trial with random forest

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [40]:
# Nested cross-validation to try different split into train and test set
rnd_clsf = RandomForestClassifier(n_jobs=-1)

param_grid = {
    "n_estimators": np.arange(2, 100, 1),
    "max_depth": np.arange(1, 5, 1),
    "min_samples_leaf": np.arange(2, 30, 1),
    "min_samples_split": np.arange(2, 30, 1),
    }

NUM_TRIALS = 15
rnd_srch_results_red = {}
for i in range(NUM_TRIALS):
     cv_split = StratifiedKFold(n_splits=3, shuffle=True, random_state=i)
     rnd_srch = RandomizedSearchCV(rnd_clsf, param_grid, cv=cv_split, n_iter=30,
                                  scoring='f1', return_train_score=True, verbose=0)
     rnd_srch.fit(dataset_prep_red, labels)
     rnd_srch_results_red[rnd_srch.best_estimator_] = rnd_srch.best_score_
     print("Iteration number", str(i), "completed.")

Iteration number 0 completed.
Iteration number 1 completed.
Iteration number 2 completed.
Iteration number 3 completed.
Iteration number 4 completed.
Iteration number 5 completed.
Iteration number 6 completed.
Iteration number 7 completed.
Iteration number 8 completed.
Iteration number 9 completed.
Iteration number 10 completed.
Iteration number 11 completed.
Iteration number 12 completed.
Iteration number 13 completed.
Iteration number 14 completed.


In [42]:
best = rnd_srch.best_estimator_
best

RandomForestClassifier(max_depth=2, min_samples_leaf=11, min_samples_split=22,
                       n_estimators=37, n_jobs=-1)

In [44]:
best.predict(dataset_prep_red)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [45]:
rnd_srch.best_score_

0.7222222222222222

In [46]:
for attr, coef in zip(attributes, best.feature_importances_.tolist()):
    print(coef, attr)

0.6666666666666666 PastureSurface
0.0 CattlePercentage
0.3333333333333333 FarmerSince
0.0 PercentRentedLand


In [47]:
test_classifier(best, dataset_prep_red, labels)

Cross validation scores
F1 score mean: 0.7222222222222222
precision mean: 0.5666666666666668
recall mean: 1.0

Prediction on training set results
F1 score: 0.7234042553191489
Precision: 0.5666666666666667
Recall: 1.0
